In [1]:
from __future__ import absolute_import, division, print_function
import sys
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def leaky_relu_6(x):

    if x >=0.0 and x <= 6.0:
        return x*1.0
    elif x > 6.0:
        return 6.0
    else:
        return 0.2 * x

In [3]:
np_leaky_relu_6= np.vectorize(leaky_relu_6)

In [4]:
def d_leaky_relu_6(x):
    if x >=0.0 and x < 6.0:
        return 1.0
    elif x > 6.0:
        return 0.0
    else:
        return 0.2
np_d_leaky_relu_6 = np.vectorize(d_leaky_relu_6)

In [5]:
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):

    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+2))

    tf.RegisterGradient(rnd_name)(grad)
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)

In [6]:
def relu_grad(op, grad):
    x = op.inputs[0]

    n_gr = tf_d_leaky_relu_6(x)    #defining the gradient.
    return grad * n_gr  

In [7]:

    
np_leaky_relu_6_32 = lambda x: np_leaky_relu_6(x).astype(np.float32)

def tf_leaky_relu_6(x,name=None):
    with tf.name_scope(name, "leaky_relu_6", [x]) as name:

        y = py_func(np_leaky_relu_6_32,   #forward pass function
                        [x],
                        [tf.float32],
                        name=name,
                         grad= relu_grad) #the function that overrides gradient
        y[0].set_shape(x.get_shape())     #when using with the code, it is used to specify the rank of the input.
        return y[0]

In [8]:
np_d_leaky_relu_6_32 = lambda x: np_d_leaky_relu_6(x).astype(np.float32)

def tf_d_leaky_relu_6(x,name=None):
    with tf.name_scope(name, "d_leaky_relu_6", [x]) as name:
        y = tf.py_func(np_d_leaky_relu_6_32,
                        [x],
                        [tf.float32],
                        name=name,
                        stateful=False)
        return y[0]

In [9]:
"""with tf.Session() as sess:

    x = tf.constant([[0.2,0.7],[1.2,-8.7]])
    y = tf_leaky_relu_6(x)
    tf.initialize_all_variables().run()

    print(x.eval(), y.eval(), tf.gradients(y, [x])[0].eval())"""

'with tf.Session() as sess:\n\n    x = tf.constant([[0.2,0.7],[1.2,-8.7]])\n    y = tf_leaky_relu_6(x)\n    tf.initialize_all_variables().run()\n\n    print(x.eval(), y.eval(), tf.gradients(y, [x])[0].eval())'

In [10]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  #conv1_act = tf.placeholder(tf.float32, [4,])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  #conv1_act = tf_leaky_relu_6(conv1)

  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same") #to try other activation function replcae relu6 by leaky_relu or relu
    
  conv2_act = tf_leaky_relu_6(conv2)
  
  pool2 = tf.layers.max_pooling2d(inputs=conv2_act, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu6) #to try other activation function replcae relu6 by leaky_relu or relu
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [11]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [12]:
# Create the Estimator# Creat 
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_model_dir': '/tmp/tmpna2demsl', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_is_chief': True, '_service': None, '_session_config': None, '_save_checkpoints_steps': None, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1db398c4a8>, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_save_summary_steps': 100}


In [13]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [14]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
# train one step and display the probabilties
#mnist_classifier.train(input_fn=train_input_fn, steps=1,hooks=[logging_hook])


mnist_classifier.train(input_fn=train_input_fn, steps=1,)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpna2demsl/model.ckpt.
INFO:tensorflow:step = 1, loss = 2.3122797
INFO:tensorflow:Loss for final step: 2.3122797.


In [15]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)
#Tensor("Shape:0", shape=(4,), dtype=int32)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpna2demsl/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/tmpna2demsl/model.ckpt.
INFO:tensorflow:step = 2, loss = 2.3002834
INFO:tensorflow:global_step/sec: 1.17395
INFO:tensorflow:step = 102, loss = 2.2808826 (85.183 sec)
INFO:tensorflow:global_step/sec: 1.23779
INFO:tensorflow:step = 202, loss = 2.277623 (80.791 sec)
INFO:tensorflow:global_step/sec: 1.15737
INFO:tensorflow:step = 302, loss = 2.260329 (86.404 sec)
INFO:tensorflow:global_step/sec: 1.18216
INFO:tensorflow:step = 402, loss = 2.2373593 (84.589 sec)
INFO:tensorflow:global_step/sec: 1.05287
INFO:tensorflow:step = 502, loss = 2.2122068 (94.978 sec)
INFO:tensorflow:global_step/sec: 1.20266
INFO:tensorflow:step = 602, loss = 2.19243 (83.149 sec)
INFO:tensorflow:global_step/sec: 1.23642
INFO:tensorflow:step = 702, loss = 2.1151078 (80.878 sec)
INFO:tensorflow:Saving checkpoints for 707 into /tmp/tmpna2demsl/model.ckpt.


In [16]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2019-07-15-15:44:59
INFO:tensorflow:Restoring parameters from /tmp/tmpna2demsl/model.ckpt-1001
INFO:tensorflow:Finished evaluation at 2019-07-15-15:45:43
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.6918, global_step = 1001, loss = 1.9385787
{'accuracy': 0.6918, 'global_step': 1001, 'loss': 1.9385787}


In [17]:
ans = []
x = []

for i in np.arange(-100, 100, 1.0):
    ans.append(leaky_relu_6(i))
    x.append(i)
    


-100.0
-99.0
-98.0
-97.0
-96.0
-95.0
-94.0
-93.0
-92.0
-91.0
-90.0
-89.0
-88.0
-87.0
-86.0
-85.0
-84.0
-83.0
-82.0
-81.0
-80.0
-79.0
-78.0
-77.0
-76.0
-75.0
-74.0
-73.0
-72.0
-71.0
-70.0
-69.0
-68.0
-67.0
-66.0
-65.0
-64.0
-63.0
-62.0
-61.0
-60.0
-59.0
-58.0
-57.0
-56.0
-55.0
-54.0
-53.0
-52.0
-51.0


In [ ]:
import matplotlib.pyplot as plt 
  

  
# plotting the points  
plt.plot(x, ans) 
  
# naming the x axis 
plt.xlabel('x - axis') 
# naming the y axis 
plt.ylabel('y - axis') 
  
# giving a title to my graph 
plt.title('Leaky Relu 6') 
  
# function to show the plot 
plt.show() 

In [ ]:
plt.show() 

In [ ]:
aview.flat = 0

plt.imshow(aview)

In [ ]:
ans

In [ ]:
x